In [ ]:
# Escenario: Eres analista en una cadena de cafeterías y te han encargado analizar las 
# ventas de la primera quincena de enero para entender el rendimiento por producto y por ciudad.

# Datos Iniciales: Comienza creando los dos DataFrames siguientes a partir de los diccionarios proporcionados.

import pandas as pd
import numpy as np

datos_ventas = {
    'Fecha': ['2023-01-01', '2023-01-01', '2023-01-02', '2023-01-02', '2023-01-03', '2023-01-08', '2023-01-08', '2023-01-09'],
    'Tienda_ID': ['T1', 'T2', 'T1', 'T2', 'T1', 'T2', 'T1', 'T2'],
    'Producto': ['Café', 'Tarta', 'Café', 'Pastel', 'Té', 'Café', 'Tarta', 'Pastel'],
    'Cantidad': [20, 5, 25, 7, 10, 18, 6, 8]
}

datos_tiendas = {
    'Tienda_ID': ['T1', 'T2'],
    'Ciudad': ['Valencia', 'Madrid']
}

datos_ventas_df = pd.DataFrame(datos_ventas)
datos_tiendas_df = pd.DataFrame(datos_tiendas)

# Objetivo del Análisis: Tu objetivo es generar tres tablas resumen para responder a las siguientes preguntas de negocio:

# ¿Cuál es la cantidad total de productos vendidos en cada ciudad?

union_datos_ventas_tiendas = pd.merge(datos_ventas_df, datos_tiendas_df, on='Tienda_ID', how='outer')
print(union_datos_ventas_tiendas)

productos_vendidos_por_ciudad = union_datos_ventas_tiendas.groupby('Ciudad')['Cantidad'].agg(['sum']).reset_index()
productos_vendidos_por_ciudad.columns = ['Ciudad', 'Productos Vendidos']

print(productos_vendidos_por_ciudad)

# ¿Cómo se distribuyen las ventas de cada producto por ciudad?

productos_por_ciudad = union_datos_ventas_tiendas.pivot_table(values = 'Cantidad', index = 'Ciudad', columns = 'Producto', aggfunc='sum', fill_value=0)
print(productos_por_ciudad)

# ¿Cuál ha sido el total de productos vendidos por semana?

# Primero para poder trabajar con fechas, hay que transformar en tipo datetime
# las fechas que se tienen en el dataframe

union_datos_ventas_tiendas['Fecha'] = pd.to_datetime(union_datos_ventas_tiendas['Fecha'])

# Posteriormente, se pone como índice la fecha, para poder trabajar con ella
# Para poder utilizar .resample(), el índice debe ser de tipo datetime

union_datos_ventas_tiendas = union_datos_ventas_tiendas.set_index('Fecha')

# Si se quiere obtener el total de productos vendidos por semana sin diferenciar ciudades,
# solo se hace resample sobre el índice fecha (índice simple)

ventas_semanales = union_datos_ventas_tiendas['Cantidad'].resample('W').sum()

# Pero si queremos analizar las ventas por semana *y* por ciudad, entonces usamos groupby + resample
# Esto genera un índice jerárquico (MultiIndex) con dos niveles: 'Ciudad' y 'Fecha'

ventas_semanales_por_ciudad = union_datos_ventas_tiendas.groupby('Ciudad').resample('W')['Cantidad'].sum()

# El resultado tiene un MultiIndex, por lo que para mejor visibilidad podemos resetear el índice
ventas_semanales_por_ciudad = ventas_semanales_por_ciudad.reset_index()

# Ahora, el DataFrame tiene columnas independientes para 'Ciudad', 'Fecha' y 'Cantidad' (ventas semanales)

print(ventas_semanales_por_ciudad)


# Tendrás que decidir qué pasos y funciones (merge, groupby, pivot_table, resample, etc.) 
# son necesarios para construir cada uno de estos resúmenes. ¡A por ello!

        Fecha Tienda_ID Producto  Cantidad    Ciudad
0  2023-01-01        T1     Café        20  Valencia
1  2023-01-02        T1     Café        25  Valencia
2  2023-01-03        T1       Té        10  Valencia
3  2023-01-08        T1    Tarta         6  Valencia
4  2023-01-01        T2    Tarta         5    Madrid
5  2023-01-02        T2   Pastel         7    Madrid
6  2023-01-08        T2     Café        18    Madrid
7  2023-01-09        T2   Pastel         8    Madrid
     Ciudad  sum
0    Madrid   38
1  Valencia   61
Producto  Café  Pastel  Tarta  Té
Ciudad                           
Madrid      18      15      5   0
Valencia    45       0      6  10
     Ciudad      Fecha  Cantidad
0    Madrid 2023-01-01         5
1    Madrid 2023-01-08        25
2    Madrid 2023-01-15         8
3  Valencia 2023-01-01        20
4  Valencia 2023-01-08        41
